In [1]:
import jmaps as jm
from datetime import datetime, timezone
import time
from jmaps import ResetCondition
import numpy as np

In [2]:
env = jm.JDict(data={'a':jm.JValue(1)})
env['o'] = 2
env['l'] = {'apples':3}
env['r'] = jm.Refer(['l', 'apples'])
env['i'] = jm.InvisibleParam({'oranges': 4})
try:
    env[1]=2
    print(env.keys())
    raise Exception("Keys must be strings")
except TypeError:
    pass

In [3]:

env.init_run(is_parent_path=True)
env.reset_usage()
print(env)
print(env.data['a'])
print(env)
if env.data['a'].used:
    raise ValueError("a should NOT be marked as used")
print(env['a'])
if not env.data['a'].used:
    raise ValueError("a should be marked as used")
print(env)


used=False data={'a': JValue(used=False, value=1), 'o': JValue(used=False, value=2), 'l': JDict(used=False, data={'apples': JValue(used=False, value=3)}), 'r': Refer(used=False, reference_list=['l', 'apples'], jparam=JValue(used=False, value=3)), 'i': InvisibleParam(used=False, jparam=JDict(used=False, data={'oranges': JValue(used=False, value=4)}))}
used=False value=1
used=False data={'a': JValue(used=False, value=1), 'o': JValue(used=False, value=2), 'l': JDict(used=False, data={'apples': JValue(used=False, value=3)}), 'r': Refer(used=False, reference_list=['l', 'apples'], jparam=JValue(used=False, value=3)), 'i': InvisibleParam(used=False, jparam=JDict(used=False, data={'oranges': JValue(used=False, value=4)}))}
1
used=False data={'a': JValue(used=True, value=1), 'o': JValue(used=False, value=2), 'l': JDict(used=False, data={'apples': JValue(used=False, value=3)}), 'r': Refer(used=False, reference_list=['l', 'apples'], jparam=JValue(used=False, value=3)), 'i': InvisibleParam(used=Fa

In [4]:

env.init_run(is_parent_path=True)
env.reset_usage()
print(env)
print(env['r'])
print(env)
if env['r'] !=3:
    raise ValueError("Reference does not point to l.apples")
if env.data['l'].data['apples'].used:
    raise ValueError("Reference accidentally triggered its root")

used=False data={'a': JValue(used=False, value=1), 'o': JValue(used=False, value=2), 'l': JDict(used=False, data={'apples': JValue(used=False, value=3)}), 'r': Refer(used=False, reference_list=['l', 'apples'], jparam=JValue(used=False, value=3)), 'i': InvisibleParam(used=False, jparam=JDict(used=False, data={'oranges': JValue(used=False, value=4)}))}
3
used=False data={'a': JValue(used=False, value=1), 'o': JValue(used=False, value=2), 'l': JDict(used=False, data={'apples': JValue(used=False, value=3)}), 'r': Refer(used=True, reference_list=['l', 'apples'], jparam=JValue(used=True, value=3)), 'i': InvisibleParam(used=False, jparam=JDict(used=False, data={'oranges': JValue(used=False, value=4)}))}


In [5]:

env.init_run(is_parent_path=True)
env.reset_usage()
env['r']
#################### Test show invisible w/o usage
show_unused=False
show_invisible=True
expected = ['r']
sql_data = env.get_sql_data(show_unused=show_unused, show_invisible=show_invisible)
sql_keys = list(sql_data.keys())
sql_keys.sort()
print(sql_keys)
if sql_keys != expected:
    raise ValueError(f"Improper marking of usage for show_unused={show_unused}, show_invisible={show_invisible}: expected {expected}, got {sql_keys}")
##############################################
if env['i']['oranges'] != 4:
    raise ValueError(f"Oranges {env['i']['oranges']} should be 4.")
show_unused=False
show_invisible=True
expected = ['i.oranges', 'r']
sql_data = env.get_sql_data(show_unused=show_unused, show_invisible=show_invisible)
sql_keys = list(sql_data.keys())
sql_keys.sort()
print(sql_keys)
if sql_keys != expected:
    raise ValueError(f"Improper marking of usage for show_unused={show_unused}, show_invisible={show_invisible}: expected {expected}, got {sql_keys}")

#############################################
show_unused=True
show_invisible=True
expected = ['a', 'i.oranges', 'l.apples', 'o', 'r']
sql_data = env.get_sql_data(show_unused=show_unused, show_invisible=show_invisible)
sql_keys = list(sql_data.keys())
sql_keys.sort()
print(sql_keys)
if sql_keys != expected:
    raise ValueError(f"Improper marking of usage for show_unused={show_unused}, show_invisible={show_invisible}: expected {expected}, got {sql_keys}")


###############################

show_unused=True
show_invisible=False
expected = ['a', 'l.apples', 'o', 'r']
sql_data = env.get_sql_data(show_unused=show_unused, show_invisible=show_invisible)
sql_keys = list(sql_data.keys())
sql_keys.sort()
print(sql_keys)
if sql_keys != expected:
    raise ValueError(f"Improper marking of usage for show_unused={show_unused}, show_invisible={show_invisible}: expected {expected}, got {sql_keys}")
    
##############################

show_unused=False
show_invisible=False
expected = ['r']
sql_data = env.get_sql_data(show_unused=show_unused, show_invisible=show_invisible)
sql_keys = list(sql_data.keys())
sql_keys.sort()
print(sql_keys)
if sql_keys != expected:
    raise ValueError(f"Improper marking of usage for show_unused={show_unused}, show_invisible={show_invisible}: expected {expected}, got {sql_keys}")

['r']
['i.oranges', 'r']
['a', 'i.oranges', 'l.apples', 'o', 'r']
['a', 'l.apples', 'o', 'r']
['r']


In [6]:

env['x'] = jm.XBuffer(datetime.now, timezone.utc, reset_condition=ResetCondition.ON_RUN_IF_PARENT_PATH)
t1 = env['x']
time.sleep(1)
t2 = env['x']
env.init_run(is_parent_path=True)
env.reset_usage()
time.sleep(1)
t3 = env['x']
env.init_run(is_parent_path=False)
env.reset_usage()
time.sleep(1)
t4 = env['x']

print(t1)
print(t2)
print(t3)
print(t4)

if t1 != t2:
    raise ValueError("The time resampled without an init run")
if t1 == t3:
    raise ValueError("The time did not get resampled on a parent init run")
if t3 != t4:
    raise ValueError("The time was resampled on a child path run")

2026-02-04 19:21:21.612852+00:00
2026-02-04 19:21:21.612852+00:00
2026-02-04 19:21:23.613728+00:00
2026-02-04 19:21:23.613728+00:00


In [7]:

env.init_run(is_parent_path=False)
env.reset_usage()
env['r']
env['x']
sql_data = env.get_sql_data(show_unused=False, show_invisible=False)
if list(env.get_sql_data(show_unused=False, show_invisible=False).keys()).sort() != ['l.apples', 'r', 'x'].sort():
    raise ValueError("Improper usage marking")
if type(sql_data['x']) != datetime:
    raise ValueError(f"XBuffer not returning datetime, instead got {type(sql_data['x'])}")

In [8]:

env['y'] = jm.YBuffer(np.linspace, 1, 5, jm.InvisibleParam(11))
print(env['y'])
sql_data = env.get_sql_data(show_unused=False, show_invisible=False)
print(sql_data)
if 'y.num' in sql_data:
    raise ValueError("The num for linspace should be invisible here")
sql_data = env.get_sql_data(show_unused=False, show_invisible=True)
print(sql_data)
if 'y' in sql_data:
    raise ValueError(f'YBuffer is not unraveling inputs, got {sql_data['y']}')

[1.  1.4 1.8 2.2 2.6 3.  3.4 3.8 4.2 4.6 5. ]
{'r': 3, 'x': datetime.datetime(2026, 2, 4, 19, 21, 23, 613728, tzinfo=datetime.timezone.utc), 'y.start': 1, 'y.stop': 5, 'y.var': 'linspace'}
{'r': 3, 'x': datetime.datetime(2026, 2, 4, 19, 21, 23, 613728, tzinfo=datetime.timezone.utc), 'y.start': 1, 'y.stop': 5, 'y.num': 11, 'y.var': 'linspace'}


In [9]:
print(jm.get_sql_type(1))
print(jm.get_sql_type(1.0))
print(jm.get_sql_type(True))
print(jm.get_sql_type('True'))

int
float
bool
str


In [10]:

sql_data = env.get_sql_data(show_unused=True, show_invisible=True)
print(sql_data)
schema = jm.get_sql_schema(sql_data)
print(schema)

{'a': 1, 'o': 2, 'l.apples': 3, 'r': 3, 'i.oranges': 4, 'x': datetime.datetime(2026, 2, 4, 19, 21, 23, 613728, tzinfo=datetime.timezone.utc), 'y.start': 1, 'y.stop': 5, 'y.num': 11, 'y.var': 'linspace'}
{'a': 'int', 'o': 'int', 'l.apples': 'int', 'r': 'int', 'i.oranges': 'int', 'x': 'datetime', 'y.start': 'int', 'y.stop': 'int', 'y.num': 'int', 'y.var': 'str'}


In [11]:
env.lock()
print(env['i']['oranges'])
try:
    env['i']['oranges'] = 3
    raise Exception("Environment is not locked properly")
except AttributeError:
    pass
env.unlock()
env['o'] = 5

4


In [12]:
env['l']['pies'] = np.pi
env.init_run(is_parent_path=True)
env.reset_usage()
t = dict(**env)
print(env.get_sql_data())


{'a': 1, 'o': 5, 'r': 3, 'x': datetime.datetime(2026, 2, 4, 19, 21, 24, 690302, tzinfo=datetime.timezone.utc), 'y.var': 'linspace'}


In [ ]:
if env.data['o'].dtype != float:
    raise TypeError("JValue not storing dtype")
env['o'] = 4
if env.data['o'].dtype != float:
    raise TypeError("JDict setter is wiping dtype")
env.replace({'o':2})
if env.data['o'].dtype != float:
    raise TypeError("JDict replace is wiping dtype")